# OpenUVF Image Segmentation

Copyright © 2019 Southern Company Services, Inc.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
#Import Statements
import cv2 as cv
import os
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import math
import core.utils.segmentation_utils as segment
from random import sample
from PIL import Image as im


### Inputs

#### Directories
1. images_dir = string specifiying the path (local or global) to the image directory


#### Module parameters
1. module_orientation = string specifying the orientation of the module. Choices are 'vertical' or ' horizontal' 
2. module_aspect_ratio = float specifying the aspect ratio of the module, specified as ratio of longer side to shorter side


#### Function Settings
1. debug = boolean specifying if debug plots should be generated
2. itmax = int specifying the maximum amount of refinement iterations where the segmentation routine is adjusted to increase accuracy

In [2]:
# Directories
images_dir = 'workspace\module_segmentation\Sample Set\FullSize-few'


# Imaging Parameters 


# Function Settings
debug = True
debug_outputs = 'verbose'
itmax = 10
module_orientation = 'horizontal'
module_aspect_ratio = 2

### Default Plotting Parameters

In [3]:
mpl.rcParams.keys()
mpl.rcParams['axes.spines.bottom'] = False
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False

### Image Processing

In [7]:
%matplotlib 
# Define list of images in the specified directory (NEEDS EDGE HANDLING)
image_list = os.listdir(images_dir)

# Predefined global image processing parameters
resize_rows = 600    

#Processing steps
equalize_hist = False
equalize_method = 'global'
    
    
#iterate through images
it = 1
for image_name in image_list:
    
    #Log start time
    time_start = time.time()
    
    #Debug Outputs
    if debug:
        print(f'\nProcessing Image {image_name} ({it}):') 
    
    #Define image path
    image_path = images_dir + '/' + image_name
    
    #Load image
    image_original = cv.imread(image_path)
    
    #Distortion correction - POSSIBLY IMPLEMENT - May cause more distortion than it corrects
    
    
    #Resize image to nominal size to increase computation speed(and adjust the anticipated aspect ratio as well)
    image_rows, image_cols, image_channels = image_original.shape;
    aspect_ratio = image_cols/image_rows
    image_pixels = image_cols * image_rows
    image_scalar = resize_rows/image_rows
    resize_cols = round(aspect_ratio * resize_rows)
    image = cv.resize(image_original, (resize_cols, resize_rows))
    
    if debug:
        print(f'   Aspect Ratio: {aspect_ratio:.2f}')
        print(f'   Total Pixels: {image_pixels}')
    #NEED PREPROCESSING STEP WHICH ADJUSTS THE WHITE BALANCE SOMEHOW - WHEN IMAGING, WB should be constant
    
    #Convert to RGB and isolate channels
    rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    rgb_min = np.amin(rgb, axis=2)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    red = rgb[:,:,0]
    gre = rgb[:,:,1]
    blu = rgb[:,:,2]
    
    #Calculate RGB statistics from the image 
    red_mean = np.mean(red)
    red_hist = cv.calcHist([rgb], [0], None, [256], [0, 256])
    gre_mean = np.mean(gre)
    gre_hist = cv.calcHist([gre], [0], None, [256], [0, 256])
    blu_mean = np.mean(blu)
    blu_hist = cv.calcHist([blu], [0], None, [256], [0, 256])
    print(f'   RGB Means: red = {red_mean:.2f};'\
          f' blue = {blu_mean:.2f}; green = {gre_mean:.2f}')
    
    
    #Convert to HSV and isolate channels
    hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
    hue = hsv[:,:,0]
    sat = hsv[:,:,1]
    val = hsv[:,:,2]
    
    #Calculate HSV statistics from the image
    #hue = hue.astype(np.float64)
    #hue[hue < 48] = hue + 48
    hue_mean = np.mean(hue)
    hue_hist = cv.calcHist([hue], [0], None, [256], [0, 256])
    sat_mean = np.mean(sat)
    sat_hist = cv.calcHist([sat], [0], None, [256], [0, 256])
    val_mean = np.mean(val)
    val_hist = cv.calcHist([val], [0], None, [256], [0, 256])
    print(f'   HSV Means: hue = {hue_mean:.2f};'\
          f' saturation = {sat_mean:.2f}; value = {val_mean:.2f}')

    #Detect if image is particularly dark 
    
    
    
    #Histogram Equalization
    if equalize_hist and (equalize_method == 'adaptive'):
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
        rgb_min = clahe.apply(rgb_min)
        gray = clahe.apply(gray)
        hue = clahe.apply(hue)
    elif equalize_hist and (equalize_method == 'global'):
        rgb_min = cv.equalizeHist(rgb_min)
        gray = cv.equalizeHist(gray)
        hue = cv.equalizeHist(hue)
    
    
    #Iteratively refine detected edges in the images
    if debug:
        print('   Edge Detection:')
    gauss_size = (5,5)
    gauss_std = 2;
    canny_thresh1 = 100
    canny_thresh2 = 200
    edge_it = 0
    edge_ratio_goal = 0.005 # 0.5% of total image area
    edge_error = []
    edge_repeat = True
    while edge_repeat & (edge_it < 10):
        
    
        #Adjust hue to remove influence of wrapping (hue is specified as 360 deg around a cylinder - 180 for opencv for uint8 type)
        hue_filtered = hue
        #hue_filtered[hue >= 170] = 0
        #hue_filtered[val < 48] = 0

        #Remove places with low values (brightness), apply gaussian filter, and convert to BGR 
        hue_smooth = cv.GaussianBlur(hue_filtered, gauss_size, gauss_std)
        rgb_min_smooth = cv.GaussianBlur(rgb_min, gauss_size, gauss_std)
        gray_smooth = cv.GaussianBlur(gray, gauss_size, gauss_std)
        #hue_bgr = cv.cvtColor(hue_smooth, cv.COLOR_GRAY2BGR)

        #Detect edges
        edges_rgb_min = cv.Canny(rgb_min_smooth, 0.5 * canny_thresh1, 0.5 * canny_thresh2)
        edges_gray = cv.Canny(gray, canny_thresh1, canny_thresh2)
        edges_hue = cv.Canny(hue_smooth, 25, 50)
        
        #Determine best edge source
        edge_ratio_rgb_min = np.sum(edges_rgb_min)/(255 * image_pixels)
        edge_ratio_gray = np.sum(edges_gray)/(255 * image_pixels)
        edge_ratio_hue = np.sum(edges_hue)/(255 * image_pixels)
            #edge_rndmns_rgb_min = 
            #edge_rndmns_gray
            #edge_rndmns_hue
        edges = edges_gray  #TEMPORARY
        edge_ratio = np.sum(edges)/(255 * image_pixels)
        edge_percent = edge_ratio * 100
        
        
        #Determine if the edge detection should be repeated and how the parameters should change
        if (edge_ratio > 0.0075) | (edge_ratio < 0.004):
            edge_repeat = True
            edge_error.append(edge_ratio - edge_ratio_goal)
            edge_P = edge_ratio/edge_ratio_goal
            edge_I = 0            
            edge_D = 0
            gauss_std = (edge_ratio/edge_ratio_goal) * gauss_std 
            canny_thresh1 = (np.sqrt(edge_ratio/edge_ratio_goal) * canny_thresh1)
            canny_thresh2 = (np.sqrt(edge_ratio/edge_ratio_goal) * canny_thresh2)
            print(f'      ({edge_it}): Ratio = {edge_percent:.2f}%   -   Gauss STD = {gauss_std:.2f}; Canny Thresh = {canny_thresh1:.2f}')
        else:
            edge_repeat = False
            print(f'      ({edge_it}): Ratio = {edge_percent:.2f}%')
            
        #Iterate iteration counter
        edge_it = edge_it + 1
    
    #Detect prominent lines
    if debug:
        print('   Line Detection:')
    max_std = 1                                     #STD cutoff for line deviation
    min_line_length = 100
    max_line_gap = 10
    hough_threshold = 100
    hough_theta = np.pi/720
    hough_rho = 1
    line_it = 0
    line_repeat = True
    while line_repeat & (line_it < 10):
    
        #Allocate variables
        hough_angles = []
    
        #Calculate lines with specified parameters
        edge_lines = cv.HoughLinesP(edges, rho = hough_rho, theta = hough_theta, threshold = hough_threshold, \
                                   minLineLength = min_line_length, maxLineGap = max_line_gap)
        
        #Calculate line orientations
        if edge_lines is not None:
            for i in range(0, len(edge_lines)):
                l = edge_lines[i][0]
                ang = np.arctan((l[3] - l[1])/(l[2]-l[0])) * 180/np.pi
                hough_angles.append(ang)
            
        #Determine number of lines detected
        hough_angles = np.array(hough_angles)
        hori_angles = (hough_angles >= -45) * (hough_angles <= 45)
        hori_lines = edge_lines[hori_angles]
        hori_angles = hough_angles[hori_angles]
        vert_angles = (hough_angles < -45) + (hough_angles > 45)
        vert_lines = edge_lines[vert_angles]
        vert_angles = hough_angles[vert_angles]
        vert_angles[vert_angles <=0] = vert_angles[vert_angles <=0] + 180        
        
        #Calculate mean angles for horizontal and vertical lines 
        vert_angles_mean = np.mean(vert_angles)
        vert_angles_std = np.std(vert_angles)
        hori_angles_mean = np.mean(hori_angles)
        hori_angles_std = np.std(hori_angles)
        
        
        
        
        # Filter out invalid candidates
        reject_lines = []
#        vert_angles_valid = (vert_angles <= (vert_angles_mean + max_std*vert_angles_std)) * (vert_angles >= (vert_angles_mean - max_std*vert_angles_std))
#        vert_angles = vert_angles[vert_angles_valid]
#        reject_lines = vert_lines[~vert_angles_valid]
#        vert_lines = vert_lines[vert_angles_valid]
#        hori_angles_valid = (hori_angles <= (hori_angles_mean + max_std*hori_angles_std)) * (hori_angles >= (hori_angles_mean - max_std*hori_angles_std))
#        hori_angles = hori_angles[hori_angles_valid]
#        reject_lines = np.concatenate((reject_lines, hori_lines[~hori_angles_valid]), 0)
#        hori_lines = hori_lines[hori_angles_valid]
            
    
        #Remove redundant lines (i.e. those that are essentially overlapping)
            #print(hori_lines)
        angles_unique_thresh = 0.5 * hough_theta * 180/np.pi
        hori_angles_unique = []
        hori_lines_unique = []
        hori_angles_groups = []
        hori_lines_groups = []
        for h in range(0, len(hori_lines)):
            
            #Determine if the angle belongs to an existing group
            hori_angle = hori_angles[h]
            hori_angle_diff = [ang - hori_angle for ang in hori_angles_unique]
            unique_angles = np.any([abs(diff) < angles_unique_thresh for diff in hori_angle_diff])
            if not unique_angles:
                hori_angles_unique.append(hori_angle)
                hori_lines_unique.append(hori_lines[h])
                hori_angles_groups.append([hori_angle])
                hori_lines_groups.append([hori_lines[h]])
            else:                         #Place the line in an existing group
                min_ind = np.argmin(abs(np.asarray(hori_angle_diff)))
                hori_angles_groups[min_ind].append(hori_angle)
                hori_lines_groups[min_ind].append(hori_lines[h])
                
                
        #print(hori_angles_unique)
        #print(hori_angles_groups)
        #x=y
        #hori_angles = hori_angles_unique
        #hori_lines = hori_lines_unique
        #print(hori_lines)
        
        #Iterate through all line pairs and determine approximate vanishing points
        vert_vanish_pts = []
        vert_vanish_xs = []
        vert_vanish_ys = []
        for a in range(0, len(vert_lines)):
            line1 = vert_lines[a][0]
            for b in range(0, len(vert_lines)):
                if (b != a) and (abs(vert_angles[a] - vert_angles[b]) > 0.1):
                    line2 = vert_lines[b][0]
                    line1_m = (line1[3] - line1[1])/(line1[2] - line1[0])
                    line1_b = line1[1] - (line1_m * line1[0])
                        #print(line1), print(line1_m), print(line1_b)
                    line2_m = (line2[3] - line2[1])/(line2[2] - line2[0])
                    line2_b = line2[1] - (line2_m * line2[0])
                        #print(line2), print(line2_m), print(line2_b)
                    if (line1_m != line2_m) and (line1_b != line2_b):
                        vert_vanish_xs.append((line2_b - line1_b)/(line1_m - line2_m))
                        vert_vanish_ys.append(line1_m * vert_vanish_xs[-1] + line1_b)
                        vert_vanish_pts.append([vert_vanish_xs[a], vert_vanish_ys[a]])
                        #print(vp_x), print(vp_y)
                    #POSSIBLE DIVIDE BY ZERO ERROR
        vert_vanish_pt = [np.nanmean(vert_vanish_xs), np.nanmean(vert_vanish_ys)]
        hori_vanish_pts = []
        hori_vanish_xs = []
        hori_vanish_ys = []
        for a in range(0, len(hori_lines)):
            line1 = hori_lines[a][0]
            for b in range(0, len(hori_lines)):
                if (b != a) and (abs(hori_angles[a] - hori_angles[b]) > 0.1):
                    line2 = hori_lines[b][0]
                    line1_m = (line1[3] - line1[1])/(line1[2] - line1[0])
                    line1_b = line1[1] - (line1_m * line1[0])
                        #print(line1), print(line1_m), print(line1_b)
                    line2_m = (line2[3] - line2[1])/(line2[2] - line2[0])
                    line2_b = line2[1] - (line2_m * line2[0])
                        #print(line2), print(line2_m), print(line2_b)
                    if (line1_m != line2_m) and (line1_b != line2_b):
                        hori_vanish_xs.append((line2_b - line1_b)/(line1_m - line2_m))
                        hori_vanish_ys.append(line1_m * hori_vanish_xs[-1] + line1_b)
                        hori_vanish_pts.append([hori_vanish_xs[a], hori_vanish_ys[a]])
                        #print(vp_x), print(vp_y)
                    #POSSIBLE DIVIDE BY ZERO ERROR
        hori_vanish_pt = [np.nanmean(hori_vanish_xs), np.nanmean(hori_vanish_ys)]
            #print(hori_vanish_pt)
        
        
        #Count number of final lines
        num_hori_angles = len(hori_angles)
        num_vert_angles = len(vert_angles)
        num_lines = num_hori_angles + num_vert_angles       
        
        #Determine if the line detection procedure should be repeated and strengthen or weaken parameters to achieve this
        if (num_vert_angles < 5) or (num_hori_angles < 5):
            line_repeat = True
            min_line_length *= 0.9 
            max_line_gap *= 1.25 
            max_std *= 1.1
            print(f'      ({line_it}): {num_lines} Lines Detected: {num_hori_angles} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
                  f'- {num_vert_angles} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.) - REPEATING')
        elif (num_vert_angles > 100) or (num_hori_angles > 100):
            line_repeat = True
            min_line_length = 1.1 * min_line_length
            max_line_gap = 0.9 * max_line_gap
            max_std *= 0.75
            print(f'      ({line_it}): {num_lines} Lines Detected: {num_hori_angles} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
                  f'- {num_vert_angles} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.) - REPEATING')
        else:
            line_repeat = False
            print(f'      ({line_it}): {num_lines} Lines Detected: {num_hori_angles} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
                  f'- {num_vert_angles} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.)')
        
        #Iterate iteration counter
        line_it = line_it + 1
        
        #Failure update
        if line_repeat and (line_it == 10):
            print('      Line Detection Failed - Image Rejected')
        

    #Plot lines on the image
    if debug:
        gray_bgr = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        for i in range(0, len(vert_lines)):
            l = vert_lines[i][0]
            gray_bgr = cv.line(gray_bgr, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_8)
        for i in range(0, len(hori_lines)):
            l = hori_lines[i][0]
            gray_bgr = cv.line(gray_bgr, (l[0], l[1]), (l[2], l[3]), (0,255,0), 3, cv.LINE_8)
        for i in range(0, len(reject_lines)):
            l = reject_lines[i][0]
            gray_bgr = cv.line(gray_bgr, (l[0], l[1]), (l[2], l[3]), (255,0,0), 3, cv.LINE_8)
    
    
    #Plot vanishing points and perspective lines
    #if debug:
    #    gray_vanish_pts = gray_bgr
    #    for v in range(0,len(vert_vanish_xs)):
    #        if not (math.isnan(vert_vanish_xs[v])) and not (math.isnan(vert_vanish_ys[v])):
    #            cv.circle(gray_vanish_pts, (int(vert_vanish_xs[v]), int(vert_vanish_ys[v])), 1, (0,128,255), thickness=-1)
    #    for h in range(0,len(hori_vanish_xs)):
    #        if not (math.isnan(hori_vanish_xs[h])) and not (math.isnan(hori_vanish_ys[h])):
    #            cv.circle(gray_vanish_pts, (int(hori_vanish_xs[h]), int(hori_vanish_ys[h])), 1, (0,255,128), thickness=-1)
            
    #Select 4 representative lines - 2 Horizontal and 2 Vertical
    #vert_lines_sort = np.argsort(vert_angles)
    #vert_lines = vert_lines[vert_lines_sort]
    #hori_lines_sort = np.argsort(hori_angles)
    #hori_lines = hori_lines[hori_lines_sort]

    #PASTE HERE
    
    #Calculate mean projective transform
    print('   Projective Transform:')
    #proj_trans = segment.calculateProjectiveTransform(vert_lines, hori_lines, gray_smooth, resize_rows, image_scalar)
    
    #print(proj_trans)
    #Perform Projective Transform
    #transed = cv.warpPerspective(image_original, proj_trans, (image_cols, image_rows))
    
    #Rotation
    #rotate_trans = cv.getRotationMatrix2D(tuple(np.array([image_rows, image_cols])/2), l3_ang, 1.0)
    #transed = cv.warpAffine(transed, rotate_trans, (image_cols, image_rows))
    
    
    
    #Log end time and calculate total execution time
    time_end = time.time()
    time_execute = time_end - time_start
    print(f'   Processing time: {time_execute:0.4f} s')
    
    #Debug 
    if debug:
        
        #Create figure
        fig = plt.figure(num=f'Image {image_name} Preprocessing', tight_layout=True,figsize=[18, 9.5])
        gs = gridspec.GridSpec(4,5)
        mng = plt.get_current_fig_manager()
        
        
        
        
        
        #Calculate watershed transform
        
        
        
        # Axis RGB - Plot base image
        axrgb = fig.add_subplot(gs[0,0])
        plt.imshow(rgb)
        plt.title('RGB')
        
        # Axis R - Plot base image
        axr = fig.add_subplot(gs[0,1])
        plt.imshow(red) 
        plt.title('Red')
        
        # Axis G - Plot base image
        axg = fig.add_subplot(gs[0,2])
        plt.imshow(gre) 
        plt.title('Green')
        
        # Axis B - Plot base image
        axb = fig.add_subplot(gs[0,3])
        plt.imshow(blu) 
        plt.title('Blue')
        
        # Axis RGB Hist - Plot histograms of the three channels
        axrgbhist = fig.add_subplot(gs[0,4])
        plt.plot(red_hist, 'r-'), plt.plot(gre_hist, 'g-'), plt.plot(blu_hist, 'b-')
        plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('RGB Histograms')
        
        # Axis HSV - Plot image in hsv color space
        axhsv = fig.add_subplot(gs[1,0])
        plt.imshow(hsv)
        plt.title('HSV')
        
        # Axis H - Plot image in hsv color space
        axh = fig.add_subplot(gs[1,1])
        plt.imshow(hue)
        plt.title('Hue')
        
        # Axis S - Plot image in hsv color space
        axs = fig.add_subplot(gs[1,2])
        plt.imshow(sat)
        plt.title('Saturation')
        
        # Axis V - Plot image in hsv color space
        axv = fig.add_subplot(gs[1,3])
        plt.imshow(val)
        plt.title('Value (brightness)')
        
        # Axis Hist - Plot histograms of the three channels
        axhsvhist = fig.add_subplot(gs[1,4])
        plt.plot(hue_hist, 'k-', label = 'Hue')
        plt.plot(sat_hist, 'm-', label = 'Sat')
        plt.plot(val_hist, 'c-', label = 'Val')
        plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('HSV Histograms'), plt.legend(loc='upper left')
        
        # Axis Grau - Plot standard grayscale image
        axgray = fig.add_subplot(gs[2,0])
        plt.imshow(gray)
        plt.title('Grayscale')
        
        # Axis Gray Histogram
        axgrayhist = fig.add_subplot(gs[2,1])
        gray_hist = cv.calcHist([gray], [0], None, [256], [0, 256])
        plt.plot(gray_hist, 'k-')
        plt.title('Grayscale Histogram')
        
        # Axis RGB min - Plot standard grayscale image
        axgray = fig.add_subplot(gs[2,2])
        plt.imshow(rgb_min)
        plt.title('RGB Min (min of each channel)')
        
        # Axis RGB min Histogram
        axgrayhist = fig.add_subplot(gs[2,3])
        rgb_min_hist = cv.calcHist([rgb_min], [0], None, [256], [0, 256])
        plt.plot(rgb_min_hist, 'k-')
        plt.title('RGB min Histogram')
        
        # Axis Gaussian - Plot blurred hue image
        #axgh = fig.add_subplot(gs[3,0])
        #plt.imshow(hue)
        #plt.title('Gaussian blur')
        
        # Axis RGB min edges 
        axhe = fig.add_subplot(gs[3,0])
        plt.imshow(edges_rgb_min)
        plt.title('RGB Min Edges')
        
        # Axis Gray edges - Plot showing the edges detected in the standard grayscale image
        axhe = fig.add_subplot(gs[3,1])
        plt.imshow(edges_gray)
        plt.title('Grayscale Edges')
        
        # Axis Hue edges - Plot showing the edge
        axhe = fig.add_subplot(gs[3,2])
        plt.imshow(edges_hue)
        plt.title('Hue Edges')
        
        # Axis final edges - Plot showing the edge
        axhe = fig.add_subplot(gs[3,3])
        plt.imshow(edges)
        plt.title('Edges')
        
        # Axis Hue Lines - Plot detected lines 
        axhe = fig.add_subplot(gs[3,4])
        plt.imshow(gray_bgr)
        plt.title('Detected Lines (Hough)')
        
        #Create Second figure
        fig2 = plt.figure(num=f'Image {image_name} Segmentation', tight_layout=True,figsize=[18, 9.5])
        gs2 = gridspec.GridSpec(2,3)
        mng2 = plt.get_current_fig_manager()
        
        #Selected representative perspective lines
        axls = fig2.add_subplot(gs2[0,0])
        plt.imshow(gray_sample_lines)
        plt.title('Projective Transform Sample Lines')

        #Vanishing point plot
        axvps = fig2.add_subplot(gs2[0,1])
        plt.imshow(gray_bgr)
        
        
        
        
        plt.plot(vert_vanish_xs, vert_vanish_ys, 'b.')
        plt.plot(hori_vanish_xs, hori_vanish_ys, 'g.')
        plt.xlim(-10000, 10000), plt.ylim(-10000, 10000)
        plt.title('Vanishing Points')
        
        
            #print(vert_vanish_pts)
            #print(hori_vanish_pts)
        
        #Projective Transform Geometry Calculations Plot
        axptg = fig2.add_subplot(gs2[0,2])
        plt.imshow(gray_sample_box)
        plt.title('Projective Transform Geometry')
        
        #Projective Transform 
        axptg = fig2.add_subplot(gs2[1,0])
        #plt.imshow(transed)
        plt.title('Projective Transform')
        
        #Update figure and Pause
        plt.draw()
        fig.canvas.manager.window.raise_()
        plt.pause(0.5)
        
        #x = y
        
    #Iteration counter    
    it = it + 1
        
        
        
        
        

    

Using matplotlib backend: Qt5Agg

Processing Image DSC03022.JPG (1):
   Aspect Ratio: 1.50
   Total Pixels: 12007680
   RGB Means: red = 114.80; blue = 126.37; green = 92.68
   HSV Means: hue = 130.19; saturation = 129.30; value = 141.31
   Edge Detection:
      (0): Ratio = 0.68%
   Line Detection:


C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\ipykernel_launcher.py:183: RuntimeWarning: divide by zero encountered in int_scalars
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\ipykernel_launcher.py:260: RuntimeWarning: divide by zero encountered in int_scalars
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\ipykernel_launcher.py:264: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\ipykernel_launcher.py:257: RuntimeWarning: divide by zero encountered in int_scalars


      (0): 317 Lines Detected: 209 Horizontal (6.33 +- 0.91 deg.) - 108 Vertical (96.05 +- 2.83 deg.) - REPEATING
      (1): 255 Lines Detected: 178 Horizontal (6.39 +- 0.59 deg.) - 77 Vertical (96.10 +- 2.85 deg.) - REPEATING
      (2): 209 Lines Detected: 154 Horizontal (6.46 +- 0.52 deg.) - 55 Vertical (96.57 +- 2.78 deg.) - REPEATING
      (3): 154 Lines Detected: 121 Horizontal (6.52 +- 0.52 deg.) - 33 Vertical (96.85 +- 3.01 deg.) - REPEATING
      (4): 132 Lines Detected: 107 Horizontal (6.52 +- 0.47 deg.) - 25 Vertical (97.20 +- 3.03 deg.) - REPEATING
      (5): 90 Lines Detected: 80 Horizontal (6.50 +- 0.48 deg.) - 10 Vertical (98.93 +- 1.31 deg.)
   Projective Transform:
   Processing time: 1.8591 s


NameError: name 'gray_sample_lines' is not defined

In [ ]:
    vert_lines_samples = [vert_lines[0], vert_lines[-1]]
    hori_lines_samples = [hori_lines[0], hori_lines[-1]]
    
    #Redefine them to span entire image
    l1 = vert_lines_samples[0][0]
    l1_m = (l1[3] - l1[1])/(l1[2] - l1[0])
    l1_b = l1[1] - (l1_m * l1[0])
    l1_func = lambda y: (y - l1_b)/l1_m
    l1 = [int(l1_func(resize_rows)), resize_rows, int(l1_func(0)),0]
    
    l2 = vert_lines_samples[1][0]
    l2_m = (l2[3] - l2[1])/(l2[2] - l2[0])
    l2_b = l2[1] - (l2_m * l2[0])
    l2_func = lambda y: (y - l2_b)/l2_m
    l2 = [int(l2_func(resize_rows)), resize_rows, int(l2_func(0)),0]
    
    l3 = hori_lines_samples[0][0]
    l3_m = (l3[3] - l3[1])/(l3[2] - l3[0])
    l3_b = l3[1] - (l3_m * l3[0])
    l3_func = lambda y: (y - l3_b)/l3_m
    l3 = [int(l3_func(resize_rows)), resize_rows, int(l3_func(0)),0]
    
    l4 = hori_lines_samples[1][0]
    l4_m = (l4[3] - l4[1])/(l4[2] - l4[0])
    l4_b = l4[1] - (l4_m * l1[0])
    l4_func = lambda y: (y - l4_b)/l4_m
    l4 = [int(l4_func(resize_rows)), resize_rows, int(l4_func(0)),0]
    
    #Plot sampled perspective lines
    if debug:
        gray_sample_lines = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_sample_lines = cv.line(gray_sample_lines, (l1[0], l1[1]), (l1[2], l1[3]), (0,255,128), 3, cv.LINE_8)
        gray_sample_lines = cv.line(gray_sample_lines, (l2[0], l2[1]), (l2[2], l2[3]), (0,255,128), 3, cv.LINE_8)
        gray_sample_lines = cv.line(gray_sample_lines, (l3[0], l3[1]), (l3[2], l3[3]), (0,255,128), 3, cv.LINE_8)
        gray_sample_lines = cv.line(gray_sample_lines, (l4[0], l4[1]), (l4[2], l4[3]), (0,255,128), 3, cv.LINE_8)
     
    #Sample line geometric relationship calculations
    l1_ang = np.arctan(l1_m) * 180/np.pi
    l2_ang = np.arctan(l2_m) * 180/np.pi
    l3_ang = np.arctan(l3_m) * 180/np.pi
    l4_ang = np.arctan(l4_m) * 180/np.pi
    l1l3_x = (l3_b - l1_b)/(l1_m - l3_m)
    l1l4_x = (l4_b - l1_b)/(l1_m - l4_m)
    l2l3_x = (l3_b - l2_b)/(l2_m - l3_m)
    l2l4_x = (l4_b - l2_b)/(l2_m - l4_m)
    l1l3_y = l1_m * l1l3_x + l1_b
    l1l4_y = l1_m * l1l4_x + l1_b
    l2l3_y = l2_m * l2l3_x + l2_b
    l2l4_y = l2_m * l2l4_x + l2_b
    l1l3_pt = [l1l3_x, l1l3_y]
    l1l4_pt = [l1l4_x, l1l4_y]
    l2l3_pt = [l2l3_x, l2l3_y]
    l2l4_pt = [l2l4_x, l2l4_y]
    l1_len = np.sqrt((l1l3_pt[1] - l1l4_pt[1])**2 + (l1l3_pt[0] - l1l4_pt[0])**2)
    l2_len = np.sqrt((l2l3_pt[1] - l2l4_pt[1])**2 + (l2l3_pt[0] - l2l4_pt[0])**2)
    l3_len = np.sqrt((l1l3_pt[1] - l2l3_pt[1])**2 + (l1l3_pt[0] - l2l3_pt[0])**2)
    l4_len = np.sqrt((l1l4_pt[1] - l2l4_pt[1])**2 + (l1l4_pt[0] - l2l4_pt[0])**2)
    l1l3_ang = np.arctan((l1l3_pt[1] - l1l4_pt[1])/(l1l3_pt[0] - l1l4_pt[0])) * 180/np.pi
    l1l4_ang = np.arctan((l2l3_pt[1] - l2l4_pt[1])/(l2l3_pt[0] - l2l4_pt[0])) * 180/np.pi
    l2l3_ang = np.arctan((l1l3_pt[1] - l2l3_pt[1])/(l1l3_pt[0] - l2l3_pt[0])) * 180/np.pi
    l2l4_ang = np.arctan((l1l4_pt[1] - l2l4_pt[1])/(l1l4_pt[0] - l2l4_pt[0])) * 180/np.pi
    
    if debug:
        print('   Projective Transform:')
        gray_sample_lines = cv.circle(gray_sample_lines, (int(l1l3_x), int(l1l3_y)), 10, (255,255,255), thickness=-1)
        gray_sample_lines = cv.circle(gray_sample_lines, (int(l1l4_x), int(l1l4_y)), 10, (0,128,255), thickness=-1)
        gray_sample_lines = cv.circle(gray_sample_lines, (int(l2l3_x), int(l2l3_y)), 10, (0,128,255), thickness=-1)
        gray_sample_lines = cv.circle(gray_sample_lines, (int(l2l4_x), int(l2l4_y)), 10, (0,128,255), thickness=-1)
        
    #Redefine lines in terms of just the box that is being reshapen
    l1_box = l1l3_pt + l1l4_pt
    l2_box = l2l3_pt + l2l4_pt
    l3_box = l1l3_pt + l2l3_pt
    l4_box = l1l4_pt + l2l4_pt
    if debug:
        gray_sample_box = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_sample_box = cv.line(gray_sample_box, (int(l1_box[0]), int(l1_box[1])), (int(l1_box[2]), int(l1_box[3])), (0,255,128), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l2_box[0]), int(l2_box[1])), (int(l2_box[2]), int(l2_box[3])), (0,255,128), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l3_box[0]), int(l3_box[1])), (int(l3_box[2]), int(l3_box[3])), (0,255,128), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l4_box[0]), int(l4_box[1])), (int(l4_box[2]), int(l4_box[3])), (0,255,128), 3, cv.LINE_4)
        
        
    #Define the moving (reference) points    
    moving_pts = np.float32([[l1l3_pt], [l1l4_pt], [l2l3_pt], [l2l4_pt]])
    
    #Make second vertical line parallel to first
    l2_new_m = l1_m
    l2_new_b = l2l3_y - l2_new_m * l2l3_x
    l2l4_new_x = (l2_new_b - l4_b)/(l4_m - l2_new_m)
    l2l4_new_y = l2_new_m * l2l4_new_x + l2_new_b
    l2l4_new_pt = [l2l4_new_x, l2l4_new_y]
    l2_new_box = l2l3_pt + l2l4_new_pt
    if debug:
        gray_sample_box = cv.line(gray_sample_box, (int(l2_new_box[0]), int(l2_new_box[1])), (int(l2_new_box[2]), int(l2_new_box[3])), (0,128,256), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l2l4_x), int(l2l4_y)), (int(l2_new_box[2]), int(l2_new_box[3])), (0,128,256), 3, cv.LINE_4)
    
    #Make second horizontal line parallel to first
    l4_new_m = l3_m
    l4_new_b = l1l4_y - l4_new_m * l1l4_x
    l2l4_new_x = (l2_new_b - l4_new_b)/(l4_new_m - l2_new_m)
    l2l4_new_y = l2_new_m * l2l4_new_x + l2_new_b
    l2l4_new_pt = [l2l4_new_x, l2l4_new_y]
    l4_new_box = l1l4_pt + l2l4_new_pt
    l2_new_box = l2l3_pt + l2l4_new_pt
    if debug:
        gray_sample_box = cv.line(gray_sample_box, (int(l2_new_box[0]), int(l2_new_box[1])), (int(l2_new_box[2]), int(l2_new_box[3])), (256,128,0), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l4_new_box[0]), int(l4_new_box[1])), (int(l4_new_box[2]), int(l4_new_box[3])), (256,128,0), 3, cv.LINE_4)
    
    #Make all angles right - convert rhombus to rectangle
    if (l3_m == 0):   #Case where no rotation will be necessary at the end
        l2l4_new_x = l2l3_x
        l2l4_new_pt = [l2l4_new_x, l2l4_new_y]
        l1l4_new_x = l1l3_x
        l1l4_new_pt = [l1l4_new_x, l2l4_new_y]       
    else:
        l1_new_m = np.tan((l3_ang - 90) * (np.pi/180))
        l1_new_b = l1l3_y - l1_new_m * l1l3_x
        l1l4_new_x = (l1_new_b - l4_new_b)/(l4_new_m - l1_new_m)
        l1l4_new_y = l1_new_m * l1l4_new_x + l1_new_b
        l1l4_new_pt = [l1l4_new_x, l1l4_new_y]
        l2_new_m = l1_new_m
        l2_new_b = l2l3_y - l2_new_m * l2l3_x
        l2l4_new_x = (l2_new_b - l4_new_b)/(l4_new_m - l2_new_m)
        l2l4_new_y = l2_new_m * l2l4_new_x + l2_new_b
        l2l4_new_pt = [l2l4_new_x, l2l4_new_y]
        #l1l4_new_pt
        #l2l4_new_pt
    l1_new_box = l1l3_pt + l1l4_new_pt
    l2_new_box = l2l3_pt + l2l4_new_pt
    l3_new_box = l3_box #NO CHANGE
    l4_new_box = l1l4_new_pt + l2l4_new_pt
    
    
    if debug:
        gray_sample_box = cv.line(gray_sample_box, (int(l1_new_box[0]), int(l1_new_box[1])), (int(l1_new_box[2]), int(l1_new_box[3])), (128,0,256), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l2_new_box[0]), int(l2_new_box[1])), (int(l2_new_box[2]), int(l2_new_box[3])), (128,0,256), 3, cv.LINE_4)
        gray_sample_box = cv.line(gray_sample_box, (int(l4_new_box[0]), int(l4_new_box[1])), (int(l4_new_box[2]), int(l4_new_box[3])), (128,0,256), 3, cv.LINE_4)
    
    
    
    
    
    #Projective Transformation
    moving_pts = (1/image_scalar) * moving_pts
    fixed_pts = (1/image_scalar) * np.float32([[l1l3_pt], [l1l4_new_pt], [l2l3_pt], [l2l4_new_pt]]) 
    proj_trans = cv.getPerspectiveTransform(moving_pts, fixed_pts)